# 05. Going Modular: Part 2 (script mode)

## 1. Get data

In [6]:
import os

from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

## 2 Create Datasets and DataLoaders (script mode)

Let's use the Jupyter magic function to create a `.py` file for creating `DataLoader`s

We can save a code cell's contents to a file using the Jupyter `%%writefile filename`

In [ ]:
# Create a dictionary going_modular scripts
import os
os.makedirs("going_modular")

In [5]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader's for image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = 1
BATCH_SIZE = 32

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int = BATCH_SIZE,
    num_workers: int = NUM_WORKERS
):
    """Creates training and testing DataLoaders
    
    Takes in a training directory and testing directory path and turns them into
    PyTorch Datasets and then into PyTorch DataLoaders
    
    Args:
        train_dir: Path to training directory
        test_dir: Path to testing directory
        transform: torchvision.transforms to perform on training and testing data
        batch_size: Number of samples per batch in each of the DataLoaders
        num_workeres: Integer for number of workers per DataLoader
    
    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names) where class_names
        is a list of the target classes
        
    Example usage:
        ```
        train_dataloader, test_dataloader, class_names = create_dataloader(
            train_dir=path/to/train_dir,
            test_dir=path/to/test_dir,
            trainsform=some_transform,
            batch_size=32,
            num_workers=4
        )
        ```
    """
    
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(
        root=train_dir,
        transform=transform,
        target_transform=None,
    )
    test_data = datasets.ImageFolder(
        root=test_dir,
        transform=transform,
        target_transform=None,
    )
    
    # Get class names as a list
    class_names = train_data.classes
    
    # Turn train and test Datasets into DataLoaders
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=True,
    )
    
    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [7]:
from torchvision import transforms

data_transform = transforms.Compose([ 
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [8]:
from going_modular import data_setup

test_dataloader, train_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
)

test_dataloader, train_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x1164b7380>,
 ['pizza', 'steak', 'sushi'])

In [9]:
len(test_dataloader), len(train_dataloader)

(8, 3)

## 3. Making a model (TinyVGG) (script mode)

Let's turn out model budiling code into a Python script we can import

In [10]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model from CNN Explainer website
"""

import torch

from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch  

    Args:
        input_shape: An integer indicating number of input channels
        hidden_units: An integer indicating number of hidden units between layers
        output_shape: An integer indicating number of output units
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3,
                    stride=1,
                    padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

Writing going_modular/model_builder.py


In [12]:
import torch
from going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else (
    "mps" if torch.mps.is_available() else "cpu"
)

# Instantiate a mode from the model_builder.py script
model_1 = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=16,
    output_shape=len(class_names)
).to(device)

model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2704, out_features=3, bias=True)
  )
)

In [13]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_1.eval()
with torch.inference_mode():
    pred = model_1(img_single.to(device))
    
# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

/Users/mchojna/Documents/GitHub/pytorch-course/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0110, -0.0290,  0.0405]], device='mps:0')

Output prediction probabilities:
tensor([[0.3344, 0.3213, 0.3444]], device='mps:0')

Output prediction label:
tensor([2], device='mps:0')

Actual label:
0
